# Опис домашнього завдання

# Завдання 1

Додати метод delete для видалення пар ключ-значення таблиці HashTable , яка реалізована в конспекті.

In [1]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [[] for _ in range(self.size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        key_hash = self.hash_function(key)
        key_value = [key, value]

        if self.table[key_hash] is None:
            self.table[key_hash] = list([key_value])
            return True
        else:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    pair[1] = value
                    return True
            self.table[key_hash].append(key_value)
            return True

    def get(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    return pair[1]
        return None
    
    def delete(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for i, pair in enumerate(self.table[key_hash]):
                if pair[0] == key:
                    del self.table[key_hash][i]
                    return True
        return False
    
    def all(self):
        return [sorted(bucket) for bucket in self.table if bucket]

# Тестую хеш-таблицю:
H = HashTable(5)
H.insert("apple", 10)
H.insert("orange", 20)
H.insert("banana", 30)

print(H.all())

H.delete("apple")
print(H.get("apple"))

H.delete("orange")
print(H.get("orange"))  

H.delete("banana")
print(H.get("banana"))

H.all()

[[['orange', 20]], [['banana', 30]], [['apple', 10]]]
None
None
None


[]

# Завдання 2

Реалізувати двійковий пошук для відсортованого масиву з дробовими числами.

Написана функція для двійкового пошуку повинна повертати кортеж, де першим елементом є кількість ітерацій, потрібних для знаходження елемента.
 
Другим елементом має бути "верхня межа" — це найменший елемент, який є більшим або рівним заданому значенню.

Критерії прийняття: Виконання коду повертає кортеж, де першим елементом є кількість ітерацій, потрібних для знаходження елемента. Другим елементом є "верхня межа" (найменший елемент, який є більшим або рівним заданому значенню).

In [2]:
def binary_search(arr, x):
    low_element = 0
    high = len(arr) - 1
    mid = 0
    search_steps = 0  # ініціалізую змінну для кількості ітерацій
    

    while low_element <= high:              # починаю пошук, поки нижня межа не перевищить верхню
        search_steps += 1                   # збільшую кількість ітерацій при кожній ітерації
        mid = (high + low_element) // 2     # знаходжу середину поточного діапазону
   
        # якщо шукане значення "х" більше за значення посередині списку, ігнорую ліву половину
        if arr[mid] < x:
            low_element = mid + 1
        # якщо шукане значення "х" менше за значення посередині списку, ігнорую праву половину
        elif arr[mid] > x:
            high = mid - 1
        # якщо значення знайдено, повертаю його
        else:
            return search_steps, arr[mid]
 
# якщо елемент не знайдений
    if low_element < len(arr):
        upper_limit = arr[low_element]
    else:
        upper_limit = None

    return search_steps, upper_limit

arr = [2.5, 3.4, 4.9, 10.12, 40.25]
x = 3.4

# виводжу результати пошуку
search_steps, upper_limit = binary_search(arr, x)
print("Iterations:", search_steps)
print("Upper limit:", upper_limit)

Iterations: 4
Upper limit: 1.5


# Завдання 3

Порівняти ефективність алгоритмів пошуку підрядка: Боєра-Мура, Кнута-Морріса-Пратта та Рабіна-Карпа на основі двох текстових файлів (стаття 1, стаття 2). 

Використовуючи timeit, треба виміряти час виконання кожного алгоритму для двох видів підрядків: одного, що дійсно існує в тексті, та іншого — вигаданого (вибір підрядків за вашим бажанням). 

На основі отриманих даних визначити найшвидший алгоритм для кожного тексту окремо та в цілому.

In [2]:
from collections import defaultdict

def build_shift_table(pattern):  # створює таблицю зсувів символів у патерні для оптимізації пошуку
    """
    Створення таблиці зсувів для алгоритму Боєра-Мура.
    Пояснення змінних:
    pattern: підрядок, який потрібно знайти у тексті.
    shift_table: таблиця зсувів символів у патерні.
    pattern_length: довжина патерну.
    """
    shift_table = {}
    pattern_length = len(pattern)
    # Встановлення зсуву рівному довжині підрядка для кожного символу в підрядку
    for index, char in enumerate(pattern[:-1]):
        shift_table[char] = pattern_length - index - 1
    # Якщо символу немає в таблиці, зсув буде дорівнювати довжині підрядка
    shift_table.setdefault(pattern[-1], pattern_length)
    return shift_table

def boyer_moore_search(text, pattern):
    """
    Алгоритм Боєра-Мура для пошуку підрядка у тексті.
    Пояснення змінних:
    text: текст, у якому потрібно знайти підрядок.
    pattern: підрядок, який потрібно знайти у тексті.
    shift_table: таблиця зсувів символів у патерні.
    text_index: поточний індекс у тексті.
    pattern_index: поточний індекс символу у патерні під час перевірки на відповідність.
    """
    # Створення таблиці зсувів для патерну (підрядка)
    shift_table = build_shift_table(pattern)
    text_index = 0  # Ініціалізуємо початковий індекс для основного тексту

    # Проходження по основному тексту, порівнюючи з підрядком
    while text_index <= len(text) - len(pattern):  # головний цикл, що перевіряє можливі зсуви патерну відносно тексту
        pattern_index = len(pattern) - 1  # Починаємо з кінця підрядка

        # Порівняння символів від кінця підрядка до його початку
        while pattern_index >= 0 and text[text_index + pattern_index] == pattern[pattern_index]:  # перевірка відповідності патерну підрядку тексту
            pattern_index -= 1  # Зсуваємось до початку підрядка

        # Якщо весь підрядок збігається, повертаємо його позицію в тексті
        if pattern_index < 0:  # якщо всі символи патерну співпадають з підрядком тексту, алгоритм повертає початковий індекс відповідності
            return text_index  # Підрядок знайдено

        # Зсув індексу text_index на основі таблиці зсувів
        # Це дозволяє "перестрибувати" над неспівпадаючими частинами тексту
        text_index += shift_table.get(text[text_index + len(pattern) - 1], len(pattern))

    # Якщо підрядок не знайдено, повертаємо -1
    return -1

In [3]:
def compute_lps(pattern):
    """
    Побудова LPS (Longest Prefix Suffix) масиву для патерну.
    Пояснення змінних:
    pattern: підрядок, для якого будується LPS масив.
    lps: масив довжин префіксів, які є суфіксами.
    length: поточна довжина префіксу, який є суфіксом.
    i: поточний індекс у патерні.
    """
    lps = [0] * len(pattern)
    prefix_length = 0
    pattern_index = 1

    while pattern_index < len(pattern):
        if pattern[pattern_index] == pattern[prefix_length]:
            prefix_length += 1
            lps[pattern_index] = prefix_length
            pattern_index += 1
        else:
            if prefix_length != 0:
                prefix_length = lps[prefix_length - 1]
            else:
                lps[pattern_index] = 0
                pattern_index += 1

    return lps


def knuth_morris_pratt_search(text, pattern):
    """
    Алгоритм Кнута-Морріса-Пратта для пошуку підрядка у тексті.
    Пояснення змінних:
    text: текст, у якому потрібно знайти підрядок.
    pattern: підрядок, який потрібно знайти у тексті.
    lps: масив довжин префіксів, які є суфіксами для патерну.
    text_index: поточний індекс у тексті.
    pattern_index: поточний індекс символу у патерні під час перевірки на відповідність.
    """
    text_length = len(text)
    pattern_length = len(pattern)
    lps = compute_lps(pattern)

    text_index = 0
    pattern_index = 0

    while text_index < text_length:
        if pattern[pattern_index] == text[text_index]:
            text_index += 1
            pattern_index += 1

        if pattern_index == pattern_length:
            return text_index - pattern_index  # Підрядок знайдено

        elif text_index < text_length and pattern[pattern_index] != text[text_index]:
            if pattern_index != 0:
                pattern_index = lps[pattern_index - 1]
            else:
                text_index += 1

    return -1  # Якщо підрядок не знайдено

In [4]:
def polynomial_hash(s, base=256, modulus=101):
    """
    Повертає поліноміальний хеш рядка s.
    Пояснення змінних:
    s: рядок, для якого обчислюється хеш.
    base: базове число для хешування.
    modulus: просте число для запобігання переповнення.
    """
    n = len(s)
    hash_value = 0
    for i, char in enumerate(s):
        power_of_base = pow(base, n - i - 1) % modulus
        hash_value = (hash_value + ord(char) * power_of_base) % modulus
    return hash_value


def rabin_karp_search(main_string, substring):
    """
    Алгоритм Рабіна-Карпа для пошуку підрядка у тексті.
    Пояснення змінних:
    main_string: основний текст, у якому потрібно знайти підрядок.
    substring: підрядок, який потрібно знайти у тексті.
    base: базове число для хешування.
    modulus: просте число для запобігання переповнення.
    substring_length: довжина підрядка.
    main_string_length: довжина основного тексту.
    substring_hash: хеш-значення для підрядка пошуку.
    current_slice_hash: хеш-значення для поточного відрізка в основному рядку.
    h_multiplier: попереднє значення для перерахунку хешу.
    """
    # Довжини основного рядка та підрядка пошуку
    substring_length = len(substring)
    main_string_length = len(main_string)
    
    # Базове число для хешування та модуль
    base = 256 
    modulus = 101  
    
    # Хеш-значення для підрядка пошуку та поточного відрізка в основному рядку
    substring_hash = polynomial_hash(substring, base, modulus)
    current_slice_hash = polynomial_hash(main_string[:substring_length], base, modulus)
    
    # Попереднє значення для перерахунку хешу
    h_multiplier = pow(base, substring_length - 1) % modulus
    
    # Проходимо крізь основний рядок
    for i in range(main_string_length - substring_length + 1):
        if substring_hash == current_slice_hash:
            if main_string[i:i + substring_length] == substring:
                return i

        if i < main_string_length - substring_length:
            current_slice_hash = (current_slice_hash - ord(main_string[i]) * h_multiplier) % modulus
            current_slice_hash = (current_slice_hash * base + ord(main_string[i + substring_length])) % modulus
            if current_slice_hash < 0:
                current_slice_hash += modulus

    return -1

In [13]:
import chardet

def read_file_with_unknown_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
        # Додатково перевіряємо, чи визначене кодування є прийнятним
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            return file.read()
    except (UnicodeDecodeError, TypeError):
        # Якщо виникає помилка декодування або chardet не зміг визначити кодування, пробуємо інше кодування
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except UnicodeDecodeError:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        
text1 = read_file_with_unknown_encoding("source/article_1.txt")
text2 = read_file_with_unknown_encoding("source/article_2.txt")

In [26]:
# Реальні підрядки (вибрані частини тексту, які точно існують)
real_patterns = [
    text1[:50],  # Наприклад, перші 10 символів тексту
    text1[80:90],  # Вибірковий підрядок
    text2[:50],
    text2[80:90]
]

# Вигадані підрядки (створені рядки, які точно не існують у тексті)
fake_patterns = [
    'piove di nuovo',
    'завтра на роботу',
    'гооол!',
    'ходімо їсти морозиво'
]

patterns = real_patterns + fake_patterns

In [43]:
import timeit

# Тестування
def test_algorithms(algorithms, main_string, patterns):
    results = {}
    for algo_name, algo_func in algorithms.items():
        results[algo_name] = 0
        for pattern in patterns:
            # time_taken = timeit.timeit(lambda: algo_func(main_string, pattern), number=10)
            time_taken = measure_search_time(algo_func, main_string, pattern)
            results[algo_name] += time_taken
    return results


def measure_search_time(func, main_string: str, pattern: str) -> int:
    setup_code = f"""from __main__ import {func.__name__}"""
    stmt = f"{func.__name__}(main_string, pattern)"
    return timeit.timeit(
        stmt,
        setup=setup_code,
        globals={"main_string": main_string, "pattern": pattern},
        number=10,
    )

In [28]:
algorithms = {
    "Boyer-Moore": boyer_moore_search,
    "Knuth-Morris-Pratt": knuth_morris_pratt_search,
    "Rabin-Karp": rabin_karp_search,
}

In [49]:
results_text1 = test_algorithms(algorithms, text1.lower(), patterns)
for algo, timings in results_text1.items():
    print(f"Algorithm: {algo:<25} Time taken: {timings:.8f} seconds")

Algorithm: Boyer-Moore               Time taken: 0.03334680 seconds
Algorithm: Knuth-Morris-Pratt        Time taken: 0.18268130 seconds
Algorithm: Rabin-Karp                Time taken: 0.33566020 seconds


In [50]:
results_text2 = test_algorithms(algorithms, text2.lower(), patterns)
for algo, timings in results_text2.items():
    print(f"Algorithm: {algo:<25} Time taken: {timings:.8f} seconds")

Algorithm: Boyer-Moore               Time taken: 0.03962710 seconds
Algorithm: Knuth-Morris-Pratt        Time taken: 0.21693910 seconds
Algorithm: Rabin-Karp                Time taken: 0.39835510 seconds


In [53]:
from copy import deepcopy

# Bизначення найшвидшого алгоритму
results_total = deepcopy(results_text1)
for algo, timing in results_text2.items():
    results_total[algo] += timing

results_total

{'Boyer-Moore': 0.07297389997984283,
 'Knuth-Morris-Pratt': 0.3996203999995487,
 'Rabin-Karp': 0.7340153000259306}

Висновки:

У даній роботі порівнюється ефективність трьох алгоритмів пошуку підрядка: Боєра-Мура, Кнута-Морріса-Пратта та Рабіна-Карпа.\
Порівняння проводилося на основі двох текстових файлів з використанням двох видів підрядків: тих, що існують у тексті, та вигаданих.\
Найшвидшим алгоритм як для першого тексту, так і для другого виявився алгоритм Боєра-Мура (0.0333468 сек і 0.0396271 сек відповідно).\
Для визначення н0729947айшвидшого алгоритму в цілому було обраховано сумарний час виконання кожного алгоритму для обох текстів: 
- Боєра-Мура: 0.07297 сек;
- Кнута-Морріса-Пратта: 0.39962 сек;
- Рабіна-Карпа: 0.73402 сек.\
Алгоритм Боєра-Мура виявився найшвидшим для обох текстів та в цілому завдяки здатності пропускати частини тексту, що значно зменшує кількість порівнянь символів. Це робить його ефективним для пошуку підрядків у великих текстах з великими алфавітами, що підтверджується результатами даного тестування.\
У найкращому випадку алгоритм Боєра-Мура може мати лінійну часову складність O(n), де n — це довжина тексту. Це досягається, коли жоден символ підрядка не збігається з символами тексту, змушуючи алгоритм робити максимальний можливий зсув при кожній невдалій спробі знаходження збігу.
Однак у найгіршому випадку, особливо коли символи підрядка часто зустрічаються в тексті, але сам підрядок не знаходиться, часова складність може досягти O(n⋅m), де m — це довжина підрядка.\
У загальному випадку алгоритм Боєра-Мура, завдяки своїй стратегії зсуву, часто виявляється швидшим у порівнянні з багатьма іншими алгоритмами пошуку.